In [14]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json
import time
from selenium.webdriver.common.keys import Keys

### Declaração das urls e variáveis globais 

In [15]:
url_quatro_rodas = 'https://quatrorodas.abril.com.br/noticias/chevrolet-onix-vende-quase-duas-vezes-mais-que-hb20-no-primeiro-semestre/'
url_icarros = 'https://www.icarros.com.br/principal/index.jsp'
max_list_cars = 20
top_n_cars = 10
option = Options()
option.headless = True

### Busca a lista dos top 50 carros mais vendidos no primeiro semestre de 2020

In [16]:
def quatro_rodas_load_rank():
    # 
    #driver = webdriver.Firefox(options=option)
    driver = webdriver.Firefox(executable_path="geckodriver.exe", options=option)

    driver.get(url_quatro_rodas)
    table_rank = driver.find_element_by_xpath('//*[@id="post-249071"]/section/table[2]')
    quatro_rodas_html = table_rank.get_attribute('outerHTML')
    soup = BeautifulSoup(quatro_rodas_html, 'html.parser')
    table = soup.find(name='table')
    df_quatro_rodas = pd.read_html(str(table))[0]
    driver.quit()
    df_quatro_rodas = df_quatro_rodas[[1, 2]].copy()
    df_quatro_rodas = df_quatro_rodas.iloc[1:]
    df_quatro_rodas = df_quatro_rodas.to_dict('records')
    df_rqr = pd.DataFrame(columns=['rank','brand','model','total'])
    i = 1
    for x in df_quatro_rodas:
        brand_model = x[1].split()
        brand = brand_model[0]
        model = brand_model[1]
        model = brand_model[1] + ((len(brand_model) > 2) and (' ' + brand_model[2]) or '')
        if brand_model[0] == 'VW':
            brand = 'Volkswagen'
        if model == 'Onix Plus':  # esse modelo de carro foi removido pq não possui dados suficientes
            continue
        df_rqr = df_rqr.append({'rank': i, 'brand': brand, 'model': model, 'total': x[2]}, ignore_index=True)
        i = i + 1
    return df_rqr

### Inicia a busca consultando a página inicial do icarros

In [17]:
def icarros_search_cars(url, brand, model):
    #driver = webdriver.Firefox()
    #driver = webdriver.Firefox(options=option) # run in background
    driver = webdriver.Firefox(executable_path="geckodriver.exe", options=option)
    driver.get(url_icarros)
    button_change = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[4]/div[1]/div[2]/div/span/a')
    button_change.click()
    time.sleep(1)
    radio_brazil = driver.find_element_by_xpath('//*[@id="localizacaoBR"]')
    radio_brazil.click()

    button_brand = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[2]/div[1]/div/div/div/div/button')
    button_brand.click()
    tetx_search_brand = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[2]/div[1]/div/div/div/div/div/div/input')
    tetx_search_brand.send_keys(brand)
    tetx_search_brand.send_keys(Keys.ENTER)

    #button_model = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[2]/div[2]/div/div/div/div/button')
    #button_model.click()
    time.sleep(1)
    tetx_search_model = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[2]/div[2]/div/div/div/div/div/div/input')
    time.sleep(1)
    tetx_search_model.send_keys(model)
    tetx_search_model.send_keys(Keys.ENTER)

    time.sleep(1)
    button_search = driver.find_element_by_xpath('//*[@id="buscaForm"]/div[4]/div[2]/button')
    teste = button_search.click()

    time.sleep(5)
    window_after = driver.window_handles[0]
    driver.switch_to.window(window_after)

    icarros_save_page(driver, brand, model, 1)
    icarros_next_page(driver, brand, model, 2)

### Carrega a próxima página e chama a função para salvar os dados

In [18]:
def icarros_next_page(driver, brand, model, index):
    time.sleep(4)
    print('marca: {} - modelo: {} - indice: {}'.format(brand, model, index))
    if index == 2:
        next_page = driver.find_element_by_xpath('//*[@id="conteudoLista"]/div[4]/div[2]/div[4]/div[2]/ul/li[8]/a')
    else:
        next_page = driver.find_element_by_xpath('//*[@id="conteudoLista"]/div[4]/div[2]/div[4]/div[2]/ul/li[9]/a')
    next_page.click()

    time.sleep(5)
    window_after = driver.window_handles[0]
    driver.switch_to.window(window_after)
    
    icarros_save_page(driver, brand, model, index)
    index = index + 1
    if index > max_list_cars:
        driver.quit()
    else:
        icarros_next_page(driver, brand, model, index)

### Salva os dados brutos em arquivos

In [19]:
def icarros_save_page(driver, brand, model, index):
    with open('./dados/Carro-{}-{}-{}.html'.format(brand, model, index), 'w+', encoding='utf-8') as f:
        f.write(driver.page_source)

### Carrega a lisa dos 50 top carros mais vendidos

In [12]:
dict_df_rqr = quatro_rodas_load_rank()

In [20]:
dict_df_rqr.head(10)

,rank,brand,model,total
0,1,Chevrolet,Onix,60.267
1,2,Hyundai,HB20,32.843
2,3,Ford,Ka,29.583
3,4,Volkswagen,Gol,24.827
4,5,Renault,Kwid,23.648
5,6,Fiat,Strada,22.849
6,7,Fiat,Argo,22.727
7,8,Jeep,Renegade,20.710
8,9,Volkswagen,T-Cross,20.595
9,10,Volkswagen,Polo,20.470


### Inicia o processo para salvar os dados brutos

In [21]:
i = 1
for index, row in dict_df_rqr.iterrows():
    print(row['brand'], row['model'])
    time.sleep(5)
    icarros_search_cars(url_icarros, row['brand'], row['model'])
    i = i + 1
    if i == (top_n_cars + 1):
        break

Chevrolet Onix
marca: Chevrolet - modelo: Onix - indice: 2
marca: Chevrolet - modelo: Onix - indice: 3
marca: Chevrolet - modelo: Onix - indice: 4
marca: Chevrolet - modelo: Onix - indice: 5
marca: Chevrolet - modelo: Onix - indice: 6
marca: Chevrolet - modelo: Onix - indice: 7
marca: Chevrolet - modelo: Onix - indice: 8
marca: Chevrolet - modelo: Onix - indice: 9
marca: Chevrolet - modelo: Onix - indice: 10
marca: Chevrolet - modelo: Onix - indice: 11
marca: Chevrolet - modelo: Onix - indice: 12
marca: Chevrolet - modelo: Onix - indice: 13
marca: Chevrolet - modelo: Onix - indice: 14
marca: Chevrolet - modelo: Onix - indice: 15
marca: Chevrolet - modelo: Onix - indice: 16
marca: Chevrolet - modelo: Onix - indice: 17
marca: Chevrolet - modelo: Onix - indice: 18
marca: Chevrolet - modelo: Onix - indice: 19
marca: Chevrolet - modelo: Onix - indice: 20
Hyundai HB20
marca: Hyundai - modelo: HB20 - indice: 2
marca: Hyundai - modelo: HB20 - indice: 3
marca: Hyundai - modelo: HB20 - indice: 4